In [ ]:
import sys
sys.path

import torch
import lltm_cpp
from torch_example import LLTM

In [ ]:
batch_size, input_features, state_size = 4, 30, 5
X = torch.randn(batch_size, input_features)
h = torch.randn(batch_size, state_size)
C = torch.randn(batch_size, state_size)

rnn = LLTM(input_features, state_size)

new_h, new_C = rnn(X, (h, C))